<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment2/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import utils 
import matplotlib.pyplot as plt
import numpy as np
from numpy import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/mnist_train.csv')
print(train_df.shape)

(60000, 785)


In [ ]:
test_df = pd.read_csv('/content/drive/My Drive/mnist_test.csv')
print(test_df.shape)

(10000, 785)


In [ ]:
y_train = train_df['label'].values
X_train = train_df.drop(['label'],1).values

In [ ]:
y_test = test_df['label'].values
X_test = test_df.drop(['label'],1).values

In [ ]:
print(X_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [ ]:
print(X_test.shape)
print(y_test.shape)

(10000, 784)
(10000,)


In [ ]:
X_train=X_train.reshape(60000,28,28)
X_train.shape

(60000, 28, 28)

In [ ]:
X_test=X_test.reshape(10000,28,28)
X_test.shape

(10000, 28, 28)

In [ ]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
print(type(X_train_tensor))
print(type(X_test_tensor))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [ ]:
# add a dimension to tensor to become B*C*H*W 
if len(X_train.shape)==3:
    X_train_tensor = X_train_tensor.unsqueeze(1) 
print(X_train_tensor.shape)
if len(X_test.shape)==3: 
    X_test_tensor = X_test_tensor.unsqueeze(1)
print(X_test_tensor.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [ ]:
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [ ]:
from torch.utils.data import TensorDataset
# wrap tensors into a dataset
train_ds = TensorDataset(X_train_tensor, y_train_tensor) 
test_ds = TensorDataset(X_test_tensor, y_test_tensor)
for x,y in train_ds: 
    print(x.shape,y.item()) 
    break

torch.Size([1, 28, 28]) 5


In [ ]:
from torch.utils.data import DataLoader
# create a data loader from dataset
train_dl = DataLoader(train_ds, batch_size=8) 
test_dl = DataLoader(test_ds, batch_size=8)
            # iterate over batches
for xb,yb in train_dl:
    print(xb.shape) 
    print(yb.shape) 
    break

torch.Size([8, 1, 28, 28])
torch.Size([8])


In [ ]:
from torch import nn
# define a two-layer model 
model = nn.Sequential(
    nn.Linear(4, 5), 
    nn.ReLU(), 
    nn.Linear(5, 1),
            )
print(model)

Sequential(
  (0): Linear(in_features=4, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
)


In [ ]:
import torch.nn.functional as F
class Net(nn.Module): 
  def __init__(self):
    super(Net, self).__init__()
  def forward(self, x):
    pass

In [ ]:
def __init__(self):
  super(Net, self).__init__() 
  self.conv1 = nn.Conv2d(1, 20, 5, 1) 
  self.conv2 = nn.Conv2d(20, 50, 5, 1) 
  self.fc1 = nn.Linear(4*4*50, 500) 
  self.fc2 = nn.Linear(500, 10)

In [ ]:
def forward(self, x):
  x = F.relu(self.conv1(x))
  x = F.max_pool2d(x, 2, 2)
  x = F.relu(self.conv2(x))
  x = F.max_pool2d(x, 2, 2)
  x = x.view(-1, 4*4*50)
  x = F.relu(self.fc1(x))
  x = self.fc2(x)
  return F.log_softmax(x, dim=1)

In [ ]:
Net.__init__ = __init__
Net.forward = forward

In [ ]:
model = Net() 
print(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [ ]:
print(next(model.parameters()).device)

cpu


In [ ]:
device = torch.device("cuda:0") 
model.to(device)
print(next(model.parameters()).device) 

cuda:0


In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
summary(model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 24, 24]             520
            Conv2d-2             [-1, 50, 8, 8]          25,050
            Linear-3                  [-1, 500]         400,500
            Linear-4                   [-1, 10]           5,010
Total params: 431,080
Trainable params: 431,080
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 1.64
Estimated Total Size (MB): 1.76
----------------------------------------------------------------


Defining the loss function

In [ ]:
from torch import nn
loss_func = nn.NLLLoss(reduction="sum")

In [ ]:
for xb, yb in train_dl:
# move batch to cuda device 
  xb=xb.type(torch.float).to(device) 
  yb=yb.to(device)
# get model output
  out=model(xb)
# calculate loss value
  loss = loss_func(out, yb)
  print (loss.item())
  break

114.11956787109375


In [ ]:
# compute gradients 
loss.backward()

In [ ]:
from torch import optim
opt = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# update model parameters 
opt.step()

In [ ]:
# set gradients to zero 
opt.zero_grad()

training and evaluation

In [ ]:
def loss_batch(loss_func, xb, yb,yb_h, opt=None): 
  loss = loss_func(yb_h, yb)
  metric_b = metrics_batch(yb,yb_h)  
  if opt is not None:
    loss.backward() 
    opt.step() 
    opt.zero_grad()
  return loss.item(), metric_b

In [ ]:
def metrics_batch(target, output):
  pred = output.argmax(dim=1, keepdim=True)
  corrects=pred.eq(target.view_as(pred)).sum().item() 
  return corrects

In [ ]:
def loss_epoch(model,loss_func,dataset_dl,opt=None): 
  loss=0.0
  metric=0.0 
  len_data=len(dataset_dl.dataset) 
  for xb, yb in dataset_dl:
    xb=xb.type(torch.float).to(device) 
    yb=yb.to(device)
    yb_h=model(xb)
    loss_b,metric_b=loss_batch(loss_func, xb, yb,yb_h, opt) 
    loss+=loss_b
    if metric_b is not None:
        metric+=metric_b 
  loss/=len_data
  metric/=len_data 
  return loss, metric

In [ ]:
def train_val(epochs, model, loss_func, opt, train_dl, test_dl):
    for epoch in range(epochs):
        model.train()
        train_loss, train_metric=loss_epoch(model,loss_func,train_dl,opt)
        model.eval()
        with torch.no_grad():
          val_loss, val_metric=loss_epoch(model,loss_func,test_dl) 
          accuracy=100*val_metric
          print("epoch: %d, train loss: %.6f, val loss: %.6f, accuracy: %.2f" %(epoch, train_loss,val_loss,accuracy))

In [ ]:
num_epochs=5
train_val(num_epochs, model, loss_func, opt, train_dl, test_dl)

epoch: 0, train loss: 0.049064, val loss: 0.046259, accuracy: 98.60
epoch: 1, train loss: 0.026580, val loss: 0.053161, accuracy: 98.51
epoch: 2, train loss: 0.018991, val loss: 0.060113, accuracy: 98.44
epoch: 3, train loss: 0.014824, val loss: 0.044193, accuracy: 99.02
epoch: 4, train loss: 0.012774, val loss: 0.081759, accuracy: 98.50
